# Chapter 2: Our First Model

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

## Setting up DataLoaders

We'll use the built-in dataset of `torchvision.datasets.ImageFolder` to quickly set up some dataloaders of downloaded cat and fish images.

`check_image`  is a quick little function that is passed to the `is_valid_file` parameter in the ImageFolder and will do a sanity check to make sure PIL can actually open the file. We're going to use this in lieu of cleaning up the downloaded dataset.


In [16]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

Set up the transforms for every image:

* Resize to 64x64
* Convert to tensor
* Normalize using ImageNet mean & std


In [17]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ]
)

### **데이터(Zip)를 다운로드하여 colab에 파일구조에 따라 저장하기**

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
zip_path ='/content/drive/MyDrive/Deep Learning/beginners-pytorch-deep-learning-master/chapter2_Image Classification with PyTorch/images.zip'

In [20]:
import os
import zipfile
import shutil
import random

# 1. 환경 설정
base_path = '/content'
categories = ['cat', 'fish']
splits = ['train', 'val', 'test']
# zip_path = '/content/images.zip'  # zip 파일 직접 upload 하는 경우
temp_extract_path = '/content/temp_images'

# 2. 최종 목적지 디렉토리 생성
for s in splits:
    for c in categories:
        os.makedirs(os.path.join(base_path, s, c), exist_ok=True)

# 3. 압축 해제
zip_file_extracted = False
if os.path.exists(zip_path):
    # 기존 임시 폴더가 있으면 삭제 후 생성
    if os.path.exists(temp_extract_path):
        shutil.rmtree(temp_extract_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_extract_path)
    print("✅ 1. 압축 해제 완료")
    zip_file_extracted = True
else:
    print(f"❌ Error: {zip_path} 파일을 찾을 수 없습니다.")

# 4. 실제 cat, fish 폴더의 위치를 찾는 함수
def find_category_path(root_path, target_category):
    for root, dirs, files in os.walk(root_path):
        if target_category in dirs:
            return os.path.join(root, target_category)
    return None

# 5. 폴더 기반 분류 및 이동 (zip 파일이 성공적으로 추출된 경우에만 진행)
if zip_file_extracted:
    for category in categories:
        # 임시 폴더 내부 어디든 'cat' 또는 'fish' 폴더가 있는지 검색
        src_category_dir = find_category_path(temp_extract_path, category)

        if src_category_dir is None:
            print(f"⚠️ 경고: 압축 파일 내에서 '{category}' 폴더를 찾을 수 없습니다.")
            continue
        else:
            print(f"📂 '{category}' 폴더 발견: {src_category_dir}")

        # 해당 폴더 내의 이미지 파일 목록 가져오기
        all_files = [f for f in os.listdir(src_category_dir)
                     if os.path.isfile(os.path.join(src_category_dir, f)) and not f.startswith('.')]

        random.shuffle(all_files)

        # 분할 비율 (8:1:1)
        train_idx = int(len(all_files) * 0.8)
        val_idx = int(len(all_files) * 0.9)

        # 파일 이동
        for i, f in enumerate(all_files):
            src_path = os.path.join(src_category_dir, f)
            if i < train_idx:
                dst_path = os.path.join(base_path, 'train', category, f)
            elif i < val_idx:
                dst_path = os.path.join(base_path, 'val', category, f)
            else:
                dst_path = os.path.join(base_path, 'test', category, f)
            shutil.move(src_path, dst_path)

# 6. 임시 폴더 삭제
if os.path.exists(temp_extract_path):
    shutil.rmtree(temp_extract_path)
print("\n✨ 모든 분류 및 이동 작업이 완료되었습니다!")

# 7. 결과 확인
print("-" * 30)
for s in splits:
    for c in categories:
        count = len(os.listdir(os.path.join(base_path, s, c)))
        print(f"✅ {s}/{c} 폴더 파일 수: {count}")

✅ 1. 압축 해제 완료
📂 'cat' 폴더 발견: /content/temp_images/val/cat
📂 'fish' 폴더 발견: /content/temp_images/val/fish

✨ 모든 분류 및 이동 작업이 완료되었습니다!
------------------------------
✅ train/cat 폴더 파일 수: 80
✅ train/fish 폴더 파일 수: 44
✅ val/cat 폴더 파일 수: 10
✅ val/fish 폴더 파일 수: 5
✅ test/cat 폴더 파일 수: 10
✅ test/fish 폴더 파일 수: 6


### 파이토치(PyTorch)에서 가장 많이 사용되는 이미지 데이터 로더
- 현재 저장되어 있는 이미지파일을 로딩

In [ ]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [ ]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [ ]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

In [ ]:
val_data

Dataset ImageFolder
    Number of datapoints: 18
    Root location: ./val/
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

By default, PyTorch’s data loaders are set to a batch_size of 1. You will almost certainly want to change that.

In [ ]:
batch_size=64

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [ ]:
val_data

Dataset ImageFolder
    Number of datapoints: 18
    Root location: ./val/
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

## Our First Model, SimpleNet

SimpleNet is a very simple combination of three Linear layers and ReLu activations between them. Note that as we don't do a `softmax()` in our `forward()`, we will need to make sure we do it in our training function during the validation phase.

  🏗️ SimpleNet 코드 상세 분석

    1. 클래스 정의 및 초기화 (__init__)
    신경망의 **재료(부품)**를 준비하는 단계

        class SimpleNet(nn.Module): # nn.Module을 상속받아 파이토치 모델임을 선언
            def __init__(self):
                super(SimpleNet, self).__init__() # 부모 클래스 초기화
                # 첫 번째 층: 입력 12288개 -> 출력 84개
                # 12288은 이미지 크기가 64x64x3(RGB)일 때 (64*64*3 = 12288)
                self.fc1 = nn.Linear(12288, 84)

                # 두 번째 층: 입력 84개 -> 출력 50개
                self.fc2 = nn.Linear(84, 50)

                # 세 번째 층 (출력층): 입력 50개 -> 출력 2개
                # 분류할 대상이 'cat', 'fish' 2개이므로 최종 출력은 2개
                self.fc3 = nn.Linear(50, 2)

    2. 순전파 연산 (forward)
    데이터가 들어왔을 때 어떤 순서로 계산될지 설계도를 그리는 단계

            def forward(self, x):
                # 1. 펼치기 (Flatten)
                # x.view(-1, 12288): 3차원 이미지를 1차원 벡터로
                # batch_size가 64라면 x의 크기는 [64, 12288]
                x = x.view(-1, 12288)

                # 2. 첫 번째 층 통과 + ReLU 활성화 함수
                # ReLU는 0보다 작은 값은 0으로, 큰 값은 그대로 통과시켜 학습
                x = F.relu(self.fc1(x))
                
                # 3. 두 번째 층 통과 + ReLU
                x = F.relu(self.fc2(x))
                
                # 4. 마지막 출력층 통과
                # 결과값은 [64, 2] 크기의 텐서 (각 이미지당 cat/fish 확률 점수)
                x = self.fc3(x)
                return x

In [ ]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84) #
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)

    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
simplenet = SimpleNet()

## Create an optimizer

Here, we're just using Adam as our optimizer with a learning rate of 0.001.

In [ ]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Copy the model to GPU

Copy the model to the GPU if available.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## Training

Trains the model, copying batches to the GPU if required, calculating losses, optimizing the network and perform validation for each epoch.

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 0.71, Validation Loss: 0.73, accuracy = 0.61
Epoch: 2, Training Loss: 0.39, Validation Loss: 0.20, accuracy = 0.89
Epoch: 3, Training Loss: 0.09, Validation Loss: 0.14, accuracy = 0.94
Epoch: 4, Training Loss: 0.06, Validation Loss: 0.08, accuracy = 1.00
Epoch: 5, Training Loss: 0.03, Validation Loss: 0.04, accuracy = 1.00


## Making predictions

Labels are in alphanumeric order, so `cat` will be 0, `fish` will be 1. We'll need to transform the image and also make sure that the resulting tensor is copied to the appropriate device before applying our model to it.

In [ ]:
labels = ['cat','fish']

img = Image.open("./val/fish/DSCN0326.jpg")
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction])

fish


## Saving Models

We can either save the entire model using `save` or just the parameters using `state_dict`. Using the latter is normally preferable, as it allows you to reuse parameters even if the model's structure changes (or apply parameters from one model to another).

In [ ]:
torch.save(simplenet, "/tmp/simplenet")
simplenet = torch.load("/tmp/simplenet", weights_only=False)


In [ ]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)

<All keys matched successfully>